## Importing Libraries

In [ ]:
!pip install opencv-python

In [ ]:
import cv2
import os
from PIL import Image
import tensorflow as tf
from tensorflow import keras
import numpy as np
from sklearn.model_selection import train_test_split
from keras.utils import normalize
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Activation, Dropout, Flatten, Dense
import matplotlib.pyplot as plt

## Datasets and Labels

In [ ]:
image_directory = 'datasets/'

In [ ]:
no_tumour_images = os.listdir(image_directory + 'no/')
yes_tumour_images = os.listdir(image_directory + 'yes/')

In [ ]:
dataset = []
label = []
INPUTSIZE = 256

In [ ]:
for i, image_name in enumerate(no_tumour_images):
    if(image_name.split('.')[1] == 'jpg'):
        image = cv2.imread(image_directory + 'no/' + image_name)
        
        image = Image.fromarray(image, 'RGB')
        image = image.resize((INPUTSIZE, INPUTSIZE))

        dataset.append(np.array(image))
        label.append(0)

In [ ]:
for i, image_name in enumerate(yes_tumour_images):
    if(image_name.split('.')[1] == 'jpg'):
        image = cv2.imread(image_directory + 'yes/' + image_name)
        
        image = Image.fromarray(image, 'RGB')
        image = image.resize((INPUTSIZE, INPUTSIZE))

        dataset.append(np.array(image))
        label.append(1)

In [ ]:
dataset = np.array(dataset)
label = np.array(label)

## Data Splitting

In [ ]:
xtrain, xtest, ytrain, ytest = train_test_split(dataset, label, test_size=0.2, random_state=0)

In [ ]:
print(xtrain.shape)
print(xtest.shape)
print(ytrain.shape)
print(ytest.shape)

In [ ]:
xtrain = normalize(xtrain, axis=1)
xtest = normalize(xtest, axis=1)

## Model Build

In [ ]:
#Basic CNN Model
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape = (INPUTSIZE, INPUTSIZE, 3)))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), kernel_initializer='he_uniform'))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), kernel_initializer='he_uniform'))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Conv2D(256, (3, 3), kernel_initializer='he_uniform'))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(2, 2)))


model.add(Flatten())

model.add(Dense(256)) #corresponding to INPUTSIZE
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

#############RESNET50 Model#####################
# base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

# for layer in base_model.layers:
#     layer.trainable = False

# # Add custom top layers for binary classification
# model = Sequential([
#     base_model,
#     Flatten(),
#     Dense(128, activation='relu'),
#     Dense(1, activation='sigmoid')
# ])



###########RESNET50 Model#######################
# base_model = ResNet50(include_top=False, weights='imagenet')
# x = base_model.output
# x = GlobalAveragePooling2D()(x)
# x = Dense(1024, activation = 'relu')(x)
# predictions = Dense(train_generator.num_classes, activation='sigmoid')(x)
# model = Model(inputs=base_model.input, outputs=predictions)

# for layer in base_model.layers:
#     layer.trainable = False

In [ ]:
model.compile(loss= 'binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])

In [ ]:
model.fit(xtrain, ytrain, 
          batch_size=16, 
          verbose=1, 
          epochs=3, 
          validation_data=(xtest, ytest), 
          shuffle=False)

In [ ]:
model.save('brain_tumour_7epochs.h5')

## Model Accuracy

In [ ]:
test_loss, test_acc = model.evaluate(xtest, ytest)
print(f"This Model error is: {round((test_loss)*100, 2)}% error")
print(f"This Model scores: {round((test_acc)*100, 2)}% accuracy")

## Image Prediction

In [ ]:
import cv2
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model

In [ ]:
model = load_model('brain_tumour_7epochs.h5')

## Predicting existing image

In [ ]:
pred = model.predict(np.expand_dims(xtest[0], axis=0)).round(2) # predicting the 0th index image of xtest folder

In [ ]:
pred[0] # Predicted Value in array

In [ ]:
np.argmax(pred) # finding the argmax optimal value of all feasible predicted value

In [ ]:
ytest[0] # actual value

In [ ]:
thresh = np.argmax(pred)

In [ ]:
if thresh >= 0.5:
    print(thresh)
    print("tumour")
else:
    print(thresh)
    print("Normal")

## predicting new image from gallery

In [ ]:
import cv2
import matplotlib.pyplot as plt

In [ ]:
image = cv2.imread(r"C:\Users\SHASHANK K\pythonProject\Brain Tumour Detection\Datasets\pred\pred48.jpg")
plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
plt.show()

In [ ]:
resize = tf.image.resize(image, (INPUTSIZE, INPUTSIZE))
plt.imshow(resize.numpy().astype(int))
plt.show()

In [ ]:
pred = model.predict(np.expand_dims(resize/255, axis = 0)).round(2)

In [ ]:
pred[0]

In [ ]:
if pred[0] >= 0.5:
    print("tumour")
else:
    print("Normal")

## Graph Plot

In [ ]:
logdir='logs'
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)
hist = model.fit(xtrain, ytrain, 
          batch_size=16, 
          verbose=1, 
          epochs=3, 
          validation_data=(xtest, ytest), 
          shuffle=False)

In [ ]:
fig = plt.figure()
plt.plot(hist.history['loss'], color='teal', label='loss')

fig.suptitle('Loss', fontsize=20)
plt.legend(loc="upper left")
plt.show()

In [ ]:
fig = plt.figure()
plt.plot(hist.history['accuracy'], color='teal', label='accuracy')

fig.suptitle('Accuracy', fontsize=20)
plt.legend(loc="upper left")
plt.show()